In [1]:
%load_ext autoreload
%autoreload 2

#
# Python module import.
#

import sys
sys.path.append("../")
import math
import copy
import pickle

import torch
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np

import imodal

device = 'cpu'
torch.set_default_dtype(torch.float32)
imodal.Utilities.set_compute_backend('keops')

In [2]:
bas = imodal.Models.Deformable3DImage.load_from_file("/home/leander/data/imagen/27113512/regions/nat_amyg_000027113512_bas_roi.nii.gz", device=device, extent='match')
# bas = imodal.Models.Deformable3DImage.load_from_file("/home/leander/data/imagen/27113512/regions/nat_amyg_000027113512_bas.nii")
# fu2 = imodal.Models.Deformable3DImage.load_from_file("/home/leander/data/imagen/27113512/regions/nat_amyg_000027113512_fu2.nii")
fu3 = imodal.Models.Deformable3DImage.load_from_file("/home/leander/data/imagen/27113512/regions/nat_amyg_000027113512_fu3_roi.nii.gz", device=device, extent='match')
# fu3 = imodal.Models.Deformable3DImage.load_from_file("/home/leander/data/imagen/27113512/regions/nat_amyg_000027113512_fu3.nii")

deformed = imodal.Models.Deformable3DImage.load_from_file("/home/leander/deformed.nii.gz")

In [17]:
import matplotlib.pyplot as plt
%matplotlib qt5
plt.subplot(1, 3, 1)
plt.imshow(deformed.bitmap[:, :, 20], cmap='gray')
plt.subplot(1, 3, 2)
plt.imshow(bas.bitmap[:, :, 20], cmap='gray')
plt.subplot(1, 3, 3)
plt.imshow(fu3.bitmap[:, :, 20], cmap='gray')

plt.show()


In [28]:
%matplotlib qt5
plt.subplot(1, 2, 1)
plt.imshow(bas.bitmap[:, :, 20] - fu3.bitmap[:, :, 20], cmap='gray')
plt.subplot(1, 2, 2)
plt.imshow(deformed.bitmap[:, :, 20] - fu3.bitmap[:, :, 20], cmap='gray')
plt.show()


In [34]:
print(torch.where((deformed.bitmap - fu3.bitmap).flatten() > 0.))

(tensor([ 63637,  63718,  63719,  ..., 144900, 144901, 144902]),)


In [40]:
%matplotlib qt5
plt.hist(torch.abs((deformed.bitmap - fu3.bitmap).flatten()).numpy(), bins=torch.linspace(0., 500., 50).tolist()[1:])
plt.show()

In [6]:
bas.silent_module.to_(device=device)
fu3.silent_module.to_(device=device)

In [22]:
angles = [1.8747820e-1, 2.9930867e-2, 3.2038878e-2]
offset = [-6.6036131e-1, 1.6238220e1, -8.5417991]
# angles = [1.7675363, 4.8365262e-2, -1.6353496]
# offset = [68.525068, 25.250773, -57.444409]
# angles = [0., 0., 0.]
# offset = [0., 0., 0.]
rigid_deformation = imodal.Utilities.rigid_deformation3d(angles[::-1], offset).to(device=device)

print(rigid_deformation)
print("===")
print(fu3.affine)
print("===")
# trans = fu3.affine.float() @ rigid_deformation
trans = rigid_deformation @ fu3.affine.float()

print(trans)

np.savetxt("affine_fu3_bas.txt", trans)

points = torch.cat([fu3.silent_module.manifold.gd.detach(), torch.ones(fu3.silent_module.manifold.gd.shape[0], 1, device=device)], dim=1)
transformed_points = imodal.Utilities.linear_transform(points, rigid_deformation)[:, 0:3]
deformed_fu3 = imodal.Utilities.deformed_intensities3d(transformed_points, fu3.bitmap, extent=fu3.extent)

tensor([[ 0.9820, -0.1863, -0.0299, -0.6604],
        [ 0.1853,  0.9822, -0.0320, 16.2382],
        [ 0.0354,  0.0259,  0.9990, -8.5418],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])
===
tensor([[  0.0000,   0.0000,   1.1000, -43.0896],
        [ -1.0938,   0.0000,   0.0000,  55.4882],
        [  0.0000,   1.0938,   0.0000, -17.6701],
        [  0.0000,   0.0000,   0.0000,   1.0000]], dtype=torch.float64)
===
tensor([[ 2.0376e-01, -3.2732e-02,  1.0802e+00, -5.2785e+01],
        [-1.0742e+00, -3.5021e-02,  2.0388e-01,  6.3315e+01],
        [-2.8325e-02,  1.0927e+00,  3.8893e-02, -2.6281e+01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]])


In [89]:
print(torch.sum(bas.bitmap>0.))
print(torch.sum(fu3.bitmap>0.))
print(torch.sum(deformed_fu3>0.))

tensor(6921, device='cuda:2')
tensor(7108, device='cuda:2')
tensor(0, device='cuda:2')


In [6]:
nib.save(nib.Nifti1Image(deformed_fu3.cpu().numpy(), fu3.affine.numpy()), "deformed_fu3.nii.gz")

In [ ]:
print(bas.bitmap.shape)
# print(fu2.bitmap.shape)
print(fu3.bitmap.shape)

In [104]:
deformables_shape = bas.bitmap.shape

aabb = imodal.Utilities.AABB(0., deformables_shape[0], 0., deformables_shape[1], 0., deformables_shape[2])

# points_density = 0.005
points_density = 0.05
lddmm_points = aabb.fill_uniform_density(points_density)

In [105]:
print(lddmm_points.shape)
print(lddmm_points.dtype)

torch.Size([13175, 3])
torch.float32


In [108]:
scale = 2./points_density**(1/3)
print(scale)
lddmm = imodal.DeformationModules.ImplicitModule0(3, lddmm_points.shape[0], scale, gd=lddmm_points, nu=0.1)

5.428835233189813


In [109]:

lddmm.to_(device=device)
lddmm._ImplicitModule0_KeOps__keops_backend = 'GPU'

In [110]:
model = imodal.Models.RegistrationModel(bas, [lddmm], imodal.Attachment.L2NormAttachment())

In [ ]:
shoot_solver = 'euler'
shoot_it = 10
costs = {}
fitter = imodal.Models.Fitter(model, optimizer='torch_lbfgs')


In [112]:
fitter.fit([fu3], 100, costs=costs, options={'shoot_solver': shoot_solver, 'shoot_it': shoot_it, 'line_search_fn': 'strong_wolfe'})

Starting optimization with method torch LBFGS
Initial cost={'deformation': tensor(3466.1306, device='cuda:2'), 'attach': tensor(1.7155e+08, device='cuda:2', dtype=torch.float64)}


Evaluated model with costs=171550851.01664472


Evaluated model with costs=173251103.4619164


Evaluated model with costs=171536223.15270478


Evaluated model with costs=171522482.1886617


...

Evaluated model with costs=141620107.16627276


Evaluated model with costs=141531699.99189785


Evaluated model with costs=141439331.64222795
Time: 13014.25181510672
Iteration: 13
Costs
deformation=4979.54052734375
attach=141434352.1017006
Total cost=141439331.64222795


Evaluated model with costs=141439331.64222795


KeyboardInterrupt: 

In [118]:
fitter.fit([fu3], 200, costs=costs, options={'shoot_solver': shoot_solver, 'shoot_it': shoot_it, 'line_search_fn': 'strong_wolfe'})

Starting optimization with method torch LBFGS
Initial cost={'deformation': tensor(9819.4375, device='cuda:2'), 'attach': tensor(1.0542e+08, device='cuda:2', dtype=torch.float64)}


Evaluated model with costs=105427385.63556267


Evaluated model with costs=105422551.5510445


Evaluated model with costs=105421904.33925249


Evaluated model with costs=105415528.68379077


...

Evaluated model with costs=99763658.98054124


Evaluated model with costs=99761396.73888786


Evaluated model with costs=99756916.46350509


Evaluated model with costs=99754834.95618398


KeyboardInterrupt: 

In [119]:
with torch.autograd.no_grad():
    intermediates = {}
    # start = time.perf_counter()
    deformed_bitmap = model.compute_deformed(shoot_solver, shoot_it, intermediates=intermediates)[0][0]

In [120]:
print(torch.sum(deformed_bitmap > 0.))

tensor(8262, device='cuda:2')


In [121]:
print(deformed_bitmap.shape)

torch.Size([67, 44, 83])


In [122]:
import nibabel as nib
import numpy as np
print(bas.affine)
print(fu3.affine)
print(bas.geometry[0].cpu().detach().numpy().shape)
# nib.save(nib.Nifti1Image(bas.geometry[0].cpu().detach().numpy(), bas.affine.numpy()), "deformed.nii.gz")
nib.save(nib.Nifti1Image(deformed_bitmap.cpu().numpy(), bas.affine.numpy()), "deformed.nii.gz")


tensor([[  0.0000,   0.0000,   1.1000, -40.6946],
        [ -1.0938,   0.0000,   0.0000,  79.2919],
        [  0.0000,   1.0938,   0.0000, -26.4152],
        [  0.0000,   0.0000,   0.0000,   1.0000]], dtype=torch.float64)
tensor([[  0.0000,   0.0000,   1.1000, -43.0896],
        [ -1.0938,   0.0000,   0.0000,  55.4882],
        [  0.0000,   1.0938,   0.0000, -17.6701],
        [  0.0000,   0.0000,   0.0000,   1.0000]], dtype=torch.float64)
(67, 44, 83)
